<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Github - Get active projects
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Github/Github_Get_active_projects.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/open_in_naas.svg"/></a>

The objective of the notebook is provide a list of active projects that an organization is working on

**Tags:** #github #projects #naas_drivers

**Author:** [Sanjeet Attili](https://www.linkedin.com/in/sanjeet-attili-760bab190/)

## Input

### Imports

In [1]:
import requests
import pandas as pd
from urllib.parse import urlencode
from datetime import datetime
from naas_drivers import github

## Setup Github
**How to find your personal access token on Github?**

- First we need to create a personal access token to get the details of our organization from here: https://github.com/settings/tokens
- You will be asked to select scopes for the token. Which scopes you choose will determine what information and actions you will be able to perform against the API.
- You should be careful with the ones prefixed with write:, delete: and admin: as these might be quite destructive.
- You can find description of each scope in docs here (https://docs.github.com/en/developers/apps/building-oauth-apps/scopes-for-oauth-apps).

### Variables

In [2]:
# Github project url
PROJECT_URL = "https://github.com/orgs/jupyter-naas/projects"

# Github token
GITHUB_TOKEN = "ghp_COJiJEU4cQR4rjslUjM9Duxxxxxx"

## Model

In [3]:
df_projects = github.connect(GITHUB_TOKEN).projects.get(PROJECT_URL)

## Output

### Display result

In [4]:
df_projects.head()

,project_name,project_description,project_id,project_created_by,project_created_date,project_created_time,project_updated_date,project_updated_time,project_columns_url
0,Community Roadmap,Official Naas Community Roadmap,4,jravenel,2021-05-25,20:53:14,2022-03-31,15:24:44,https://api.github.com/projects/12537896/columns
